In [ ]:
"""
author: Miranda Lv
Date: Aug, 2023
"""

In [83]:
import os
import json
import requests
import geojsonio
import time
import geopandas as gpd
import geojson
from os.path import dirname as up
import pandas as pd

import json
import requests
from requests.auth import HTTPBasicAuth

from tqdm import tqdm

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))



In [84]:
root_path = up(os.getcwd())

year = '2021'
month = 'JanToJun'

meta_path = os.path.join(root_path, 'data/processing_data/planetMetadata/planetMeta_{}{}_Kory_edit.csv'.format(year,month))
dataDir = r"Z:\EPA_marsh_migration\PlanetImages\{}".format(year) #os.path.join(root_path, 'data/output')
df = pd.read_csv(meta_path)

In [85]:
df = df[df.Useful_Scene=='Y']

In [86]:
df

,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,cloud_percent,ground_control,gsd,heavy_haze_percent,instrument,...,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent,id,date,hour_min_sec,Useful_Scene
4,2021-01-20T15:56:30.824453Z,0,96,100,0,0,True,4.1,0,PSB.SD,...,158.0,29.7,2022-12-14T07:39:37Z,0.9,71,100,20210120_155630_82_2424,20210120,155630,Y
21,2021-03-03T15:54:10.209646Z,0,99,100,0,0,True,4.1,0,PSB.SD,...,152.0,42.2,2022-12-14T23:22:26Z,3.0,99,100,20210303_155410_20_2407,20210303,155410,Y
23,2021-03-05T15:55:35.286003Z,0,99,100,0,0,True,4.1,0,PSB.SD,...,151.8,42.8,2022-12-14T19:53:40Z,1.1,99,100,20210305_155535_28_2254,20210305,155535,Y
30,2021-03-07T15:55:43.344228Z,0,99,100,0,0,True,4.1,0,PSB.SD,...,151.6,43.7,2022-12-15T14:28:50Z,0.9,81,100,20210307_155543_34_2412,20210307,155543,Y
39,2021-03-21T15:06:31.963658Z,0,99,100,0,0,True,4.1,0,PSB.SD,...,133.7,42.7,2022-12-13T09:39:20Z,5.0,99,100,20210321_150631_96_2420,20210321,150631,Y
41,2021-03-22T15:54:04.805327Z,0,100,100,0,0,True,4.0,0,PSB.SD,...,149.3,49.4,2022-12-15T13:30:32Z,3.0,100,100,20210322_155404_80_2408,20210322,155404,Y
53,2021-04-03T15:03:36.807538Z,0,99,100,0,0,True,4.1,0,PSB.SD,...,130.5,47.5,2022-12-07T18:48:24Z,2.0,99,100,20210403_150336_80_242d,20210403,150336,Y
56,2021-04-04T15:51:07.774187Z,0,91,100,0,0,True,4.2,0,PSB.SD,...,146.8,54.4,2022-12-07T06:03:34Z,5.0,75,100,20210404_155107_77_2416,20210404,155107,Y
78,2021-05-04T15:52:53.827816Z,0,100,100,0,0,True,4.1,0,PSB.SD,...,139.5,63.9,2022-12-08T07:08:48Z,2.0,85,100,20210504_155253_82_240f,20210504,155253,Y
80,2021-05-06T15:01:54.524918Z,0,100,100,0,0,True,4.1,0,PSB.SD,...,120.2,56.8,2022-12-11T19:50:04Z,5.1,100,100,20210506_150154_52_2464,20210506,150154,Y


In [87]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = ''

# construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

# Setup Planet Data API base URL
URL = "https://api.planet.com/data/v1"

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

# Make a GET request to the Planet Data API
res = session.get(URL)

# Response status code
res.status_code

200

In [88]:
def activating_asset(sid, itemType, productKey):
    
    id_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(itemType, sid)
    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
    result = \
      requests.get(
        id_url,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    # Parse out useful links
    links = result.json()[productKey]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]

    # Request activation of the asset:
    activate_result = \
      requests.get(
        activation_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )
    
    activation_status_result = \
      requests.get(
        self_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    return activation_status_result.json()["status"]



# Create a function to download asset files
# Parameters: 
# - url (the location url)
# - filename (the filename to save it as. defaults to whatever the file is called originally)

def pl_download(sid, itemType, productKey, downloadDir, filename=None):
    
    assets_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets/'.format(itemType, sid)
    res = session.get(assets_url)
    assets = res.json()
    dataproduct = assets[productKey]
    location_url = dataproduct["location"]

    # Send a GET request to the provided location url, using your API Key for authentication
    res = requests.get(location_url, stream=True, auth=(API_KEY, ""))
    # If no filename argument is given
    if not filename:
        # Construct a filename from the API response
        if "content-disposition" in res.headers:
            filename = res.headers["content-disposition"].split("filename=")[-1].strip("'\"")
        # Construct a filename from the location url
        else:
            filename = location_url.split("=")[1][:10]
    # Save the file
    if os.path.isfile(os.path.join(downloadDir, filename)):
        return 'File {} exists in the directory'.format(filename)
    else:
        with open(os.path.join(downloadDir, filename), "wb") as f:
            for chunk in res.iter_content(chunk_size=1024):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    f.flush()

        return filename

# Download the file from an activated asset's location url
# pl_download(location_url)

In [89]:
allids = df.id.tolist()
item_type = "PSScene"
instruKey = "ortho_analytic_8b"

for sID in tqdm(allids): 
    
    id_status = activating_asset(sID, item_type, instruKey)
    if id_status == 'active':
        pl_download(sID, item_type, instruKey, dataDir, filename=None)
    else:
        print('Scene id {} is not active'.format(sID))
    

100%|███████████████████████████████████████████████████████████████████████████████| 14/14 [2:06:36<00:00, 542.58s/it]
